In [1]:
import pandas as pd

In [2]:
eval_df = pd.read_json("../outputs/predictions_checkpoint_1800.json")

In [3]:
eval_df.head()

,company_id,year,actual_label,predicted_direction,magnitude,certainty,reason,is_match,full_model_response
0,1000,1973,increase,decrease,Significant,Medium,\eSince\e\ sales\ehave\edeclined\esharply\ewit...,False,system\nYou are a helpful assistant.\nuser\nBa...
1,1000,1974,decrease,decrease,Moderate,Medium,"Despite some growth signals via asset buildup,...",True,system\nYou are a helpful assistant.\nuser\nBa...
2,1000,1975,increase,increase,Moderate,High,Rapidly increasing sales coupled with margin e...,True,system\nYou are a helpful assistant.\nuser\nBa...
3,1000,1976,increase,decrease,Moderate,Medium,\nThe projected decrease in EPS stems from ant...,False,system\nYou are a helpful assistant.\nuser\nBa...
4,1000,1977,decrease,error,Error,Error,"Error in parsing JSON: Expecting ',' delimiter...",False,system\nYou are a helpful assistant.\nuser\nBa...


In [4]:
eval_df["full_model_response"][0]

'system\nYou are a helpful assistant.\nuser\nBased on the following financial statements, assess whether EPS will increase or decrease in the next year:\n\nBelow is the Balance Sheet:\n------------------------------------------------------------------------------------------\n| Account Items                                                  | 1971 | 1972 |\n------------------------------------------------------------------------------------------\n| Total Asset                                                  | 29.330 | 19.907 |\n| Current Assets                                               | 19.688 | 11.326 |\n| Current Liabilities                                          | 8.507 | 3.990 |\n| Cash and Short-Term Investments                              | 2.557 | 2.027 |\n| Receivables                                                  | 8.398 | 4.186 |\n| Inventories                                                  | 8.187 | 4.743 |\n| Other Current Assets                               

In [7]:
eval_df.predicted_direction.value_counts()

predicted_direction
error                                   560
increase                                559
decrease                                398
<increase/decrease>                     122
moderate increase                        40
uncertain                                39
possible increase                        24
possible decrease                        12
moderate                                  8
slight increase                           6
steady                                    4
stable                                    3
stabilize                                 2
moderate decrease                         2
decreased                                 2
possible stabilization                    1
possibly decrease                         1
stabilization                             1
unknown                                   1
maintained                                1
decremented                               1
increased                                 1
slight      

In [57]:
full_text = eval_df[eval_df.predicted_direction == "error"].full_model_response.iloc[0]

In [58]:
text = full_text.split('assistant')[-1]
print(text)


### Step-by-Step Analysis

#### **Balance Sheet**

**Key Insights & Trends:**
1. **Total Assets**: Increased from $23,906K in '77 to $38,586K in '76, reflecting growth over three years.
2. **Current Assets**: Grew from $16,366K to $21,647K with increases across inventory and receivables suggesting robust operational efficiency.
3. **Long-term debt**: Jumped significantly from $4,592K to $13,583K indicating potential capital structure shift towards more long-term financing which could affect leverage levels going forward.
4. **Shareholder equity**: Increased from $12,309K to $15,625K showing growth in owners’ investment, supporting business expansion.

**Liquidity Analysis:** 
- Current ratio improved slightly but suggests adequate short term solvency despite rising liabilities.


#### **Income Statement**

**Key Insights & Trends:**
1. **Sales Growth**: From $50,325K to $66,214K signifies strong revenue performance, possibly driven by market penetration or product improvements.
2. **G

In [59]:
import re
import json

pattern = r'```json\s*\n(.*?)\n```'
    
# Find all JSON blocks
json_blocks = re.findall(pattern, text, re.DOTALL)[0]
print(json_blocks)

{
    "direction_of_eps_change": "Moderate Increase",
    "magnitude_of_change": "Dependent on successful execution of growth initiatives while optimizing cost structures and managing existing debts.",
    "certainty_of_assessment": "Medium",
    "reason": "\\nThe company’s demonstrated ability to grow revenues with expanding markets alignss with upward pressure on earnings per share. However, the balance between financial leverage and operational efficiency remains critical, affecting both the extent of such increase and its predictability."
"
}


In [60]:
from json_repair import repair_json
import json

def fix_json_with_library(broken_json_text):
    """
    Use json-repair library to fix common JSON issues.
    """
    try:
        # Attempt to repair the JSON
        fixed_json = repair_json(broken_json_text)
        return json.loads(fixed_json)
    except Exception as e:
        print(f"Could not repair JSON: {e}")
        return None

In [81]:
fix_json_with_library(json_blocks)

{'direction_of_eps_change': 'Moderate Increase',
 'magnitude_of_change': 'Dependent on successful execution of growth initiatives while optimizing cost structures and managing existing debts.',
 'certainty_of_assessment': 'Medium',
 'reason': '\\nThe company’s demonstrated ability to grow revenues with expanding markets alignss with upward pressure on earnings per share. However, the balance between financial leverage and operational efficiency remains critical, affecting both the extent of such increase and its predictability.'}

It is working...

In [ ]:
for i, row in eval_df.iterrows():
    is_error = (row.predicted_direction == "error")
    
    if not is_error:
        continue
    else:
        # Get Full Text to extract the JSON
        full_response_text = row.full_model_response
        
        # Extract the last assistant message
        last_text = full_response_text.split('assistant')[-1]
        
        # Find the JSON block
        try:
            pattern = r'```json\s*\n(.*?)\n```'
            json_block = re.findall(pattern, last_text, re.DOTALL)[0]
        except Exception as e:
            pattern = r'\{[^{}]*\}'
            json_block = re.findall(pattern, last_text, re.DOTALL)[0]
        
        # Fix the json with fix_json library
        fixed_json = fix_json_with_library(json_block)
        
        if fixed_json:
            eval_df.loc[i, 'predicted_direction'] = fixed_json['direction_of_eps_change']
            eval_df.loc[i, 'magnitude'] = fixed_json['magnitude_of_change']
            eval_df.loc[i, 'certainty'] = fixed_json['certainty_of_assessment']
            eval_df.loc[i, 'reason'] = fixed_json['reason']
            eval_df.loc[i, 'is_match'] = True is fixed_json['direction_of_eps_change'] == row['actual_label']
                    
        


KeyError: 'magnitude_of_change'

In [78]:
eval_df.loc[0, 'company_id']

1000